# Implementing a Route Planner
In this project you will use A\* search to implement a "Google-maps" style route planning algorithm.

## The Map

In [361]:
# Run this cell first!

from helpers import Map, load_map_10, load_map_40, show_map
import math

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Map Basics

In [362]:
map_10 = load_map_10()
show_map(map_10)

The map above (run the code cell if you don't see it) shows a disconnected network of 10 intersections. The two intersections on the left are connected to each other but they are not connected to the rest of the road network. This map is quite literal in its expression of distance and connectivity. On the graph above, the edge between 2 nodes(intersections) represents a literal straight road not just an abstract connection of 2 cities.

These `Map` objects have two properties you will want to use to implement A\* search: `intersections` and `roads`

**Intersections**

The `intersections` are represented as a dictionary. 

In this example, there are 10 intersections, each identified by an x,y coordinate. The coordinates are listed below. You can hover over each dot in the map above to see the intersection number.

In [363]:
map_10.intersections

{0: (0.7798606835438107, 0.6922727646627362),
 1: (0.7647837074641568, 0.3252670836724646),
 2: (0.7155217893995438, 0.20026498027300055),
 3: (0.7076566826610747, 0.3278339270610988),
 4: (0.8325506249953353, 0.02310946309985762),
 5: (0.49016747075266875, 0.5464878695400415),
 6: (0.8820353070895344, 0.6791919587749445),
 7: (0.46247219371675075, 0.6258061621642713),
 8: (0.11622158839385677, 0.11236327488812581),
 9: (0.1285377678230034, 0.3285840695698353)}

In [367]:
print(map_10.intersections[0])

(0.7798606835438107, 0.6922727646627362)


In [368]:
len(map_10.intersections)

10

In [369]:
type(map_10)

helpers.Map

In [370]:
type(map_10.roads)

list

**Roads**

The `roads` property is a list where `roads[i]` contains a list of the intersections that intersection `i` connects to.

In [371]:
# this shows that intersection 0 connects to intersections 7, 6, and 5
map_10.roads[0] 

[7, 6, 5]

In [372]:
type(map_10.roads[0])

list

In [373]:
# This shows the full connectivity of the map
map_10.roads

[[7, 6, 5],
 [4, 3, 2],
 [1, 4, 3],
 [1, 2, 5, 4],
 [1, 2, 3],
 [0, 3, 7],
 [0],
 [0, 5],
 [9],
 [8]]

In [377]:
# map_40 is a bigger map than map_10
map_40 = load_map_40()
show_map(map_40)

### Advanced Visualizations

The map above shows a network of roads which spans 40 different intersections (labeled 0 through 39). 

The `show_map` function which generated this map also takes a few optional parameters which might be useful for visualizaing the output of the search algorithm you will write.

* `start` - The "start" node for the search algorithm.
* `goal`  - The "goal" node.
* `path`  - An array of integers which corresponds to a valid sequence of intersection visits on the map.

In [378]:
map_40.roads[5]

[32, 16, 14]

In [379]:
a = (map_40.intersections[5][0]-map_40.intersections[34][0])**2 + (map_40.intersections[5][1]-map_40.intersections[34][1])**2

In [380]:
math.sqrt(a)

0.574734871592805

In [381]:
# run this code, note the effect of including the optional
# parameters in the function call.
show_map(map_40, start=5, goal=34, path=[5,16,37,12,34])

## The Algorithm
### Writing your algorithm
The algorithm you write will be responsible for generating a `path` like the one passed into `show_map` above. In fact, when called with the same map, start and goal, as above you algorithm should produce the path `[5, 16, 37, 12, 34]`

```bash
> shortest_path(map_40, 5, 34)
[5, 16, 37, 12, 34]
```

In [382]:
# Do not change this cell
# When you write your methods correctly this cell will execute
# without problems
class PathPlanner():
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        """ """
        self.map = M
        self.start= start
        self.goal = goal
        self.closedSet = self.create_closedSet() if goal != None and start != None else None
        # type(onenSet) = list or set
        self.openSet = self.create_openSet() if goal != None and start != None else None
        # type(cameFrom) = dict
        self.cameFrom = self.create_cameFrom() if goal != None and start != None else None
        # 2-dimensioanl list
        self.gScore = self.create_gScore() if goal != None and start != None else None
        # type(fScore) = list
        self.fScore = self.create_fScore() if goal != None and start != None else None
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None
        
    def get_path(self):
        """ Reconstructs path after search """
        if self.path:
            return self.path 
        else :
            self.run_search()
            return self.path
    
    def reconstruct_path(self, current):
        """ Reconstructs path after search """
        total_path = [current]
        while current in self.cameFrom.keys():
            current = self.cameFrom[current]
            total_path.append(current)
        return total_path

    def run_search(self):
        """ """
        if self.map == None:
            raise(ValueError, "Must create map before running search. Try running PathPlanner.set_map(start_node)")
        if self.goal == None:
            raise(ValueError, "Must create goal node before running search. Try running PathPlanner.set_goal(start_node)")
        if self.start == None:
            raise(ValueError, "Must create start node before running search. Try running PathPlanner.set_start(start_node)")

        self.closedSet = self.closedSet if self.closedSet != None else self.create_closedSet()
        self.openSet = self.openSet if self.openSet != None else  self.create_openSet()
        self.cameFrom = self.cameFrom if self.cameFrom != None else  self.create_cameFrom()
        self.gScore = self.gScore if self.gScore != None else  self.create_gScore()
        self.fScore = self.fScore if self.fScore != None else  self.create_fScore()
        
        
        while not self.is_open_empty():
            current = self.get_current_node()
            
            if current == self.goal:
                self.path = [x for x in reversed(self.reconstruct_path(current))]
                return self.path
            else:
                self.openSet.remove(current)
                self.closedSet.add(current)

            for neighbor in self.get_neighbors(current):
            
                if neighbor in self.closedSet:
                    continue    # Ignore the neighbor which is already evaluated.

                if not neighbor in self.openSet:    # Discover a new node
                    self.openSet.add(neighbor)
                    #self.record_best_path_to(current, neighbor)
                # The distance from start to a neighbor
                #the "dist_between" function may vary as per the solution requirements.
                if self.get_tenative_gScore(current, neighbor) >= self.get_gScore(neighbor):
                    continue        # This is not a better path.

                # This path is the best until now. Record it!
                self.record_best_path_to(current, neighbor)
        
        print("No Path Found")
        self.path = None
        return False

Create the following methods:

In [383]:
def create_closedSet(self):
    """ Creates and returns a data structure suitable to hold the set of nodes already evaluated"""
    # TODO: return a data structure suitable to hold the set of nodes already evaluated
    
    return set()

In [384]:
from heapq import *
def create_openSet(self):
    """ Creates and returns a data structure suitable to hold the set of currently discovered nodes 
    that are not evaluated yet. Initially, only the start node is known."""
    if self.start != None:
        # TODO: return a data structure suitable to hold the set of currently discovered nodes 
        # that are not evaluated yet. Make sure to include the start node.
#         mean_heap = []
#         print(self.map.intersections(self.start))
#         start_xy = (self.map.intersections[self.start])
#         heappush(mean_heap, (self.distance(self.start, self.goal), start_xy))
#         return mean_heap
        openSet = set()
        openSet.add(self.start)
        return openSet
    else:
        raise(ValueError, "Must create start node before creating an open set. Try running PathPlanner.set_start(start_node)")

In [385]:
def create_cameFrom(self):
    """Creates and returns a data structure that shows which node can most efficiently be reached from another,
    for each node."""
    # TODO: return a data structure that shows which node can most efficiently be reached from another,
    # for each node. 
    cameFrom = {}
    return cameFrom

In [386]:
def create_gScore(self):
    """Creates and returns a data structure that holds the cost of getting from the start node to that node, for each node.
    The cost of going from start to start is zero."""
    # TODO:  a data structure that holds the cost of getting from the start node to that node, for each node.
    # for each node. The cost of going from start to start is zero. The rest of the node's values should be set to infinity.
    gScore = {}
    for intersection in self.map.intersections:
        gScore[intersection] = 9999
    gScore[self.start] = 0
    return gScore

In [387]:
def create_fScore(self):
    """Creates and returns a data structure that holds the total cost of getting from the start node to the goal
    by passing by that node, for each node. That value is partly known, partly heuristic.
    For the first node, that value is completely heuristic."""
    # TODO:  a data structure that holds the total cost of getting from the start node to the goal
    # by passing by that node, for each node. That value is partly known, partly heuristic.
    # For the first node, that value is completely heuristic. The rest of the node's value should be 
    # set to infinity.
    fScore = {}
    for intersection in self.map.intersections:
        fScore[intersection] = 99999
    fScore[self.start] = self.distance(self.start, self.goal)
    return fScore

In [388]:
def _reset(self):
    """Private method used to reset the closedSet, openSet, cameFrom, gScore, fScore, and path attributes"""
    self.closedSet = None
    self.openSet = None
    self.cameFrom = None
    self.gScore = None
    self.fScore = None
    self.path = self.run_search() if self.map and self.start and self.goal else None

In [389]:
def set_map(self, M):
    """Method used to set map attribute """
    self._reset(self)
    self.start = None
    self.goal = None
    # TODO: Set map to new value. 
    self.map = M

In [390]:
def set_start(self, start):
    """Method used to set start attribute """
    self._reset(self)
    # TODO: Set start value. Remember to remove goal, closedSet, openSet, cameFrom, gScore, fScore, 
    # and path attributes' values.
    self.start = start
    self.closedSet = None
    self.openSet = None
    self.cameFrom = None
    self.gScore = None
    self.fScore = None

In [391]:
def set_goal(self, goal):
    """Method used to set goal attribute """
    self._reset(self)
    # TODO: Set goal value. 
    self.goal = goal

In [393]:
def get_current_node(self):
    """ Returns the node in the open set with the lowest value of f(node)."""
    # TODO: Return the node in the open set with the lowest value of f(node).
    tempDict = {}
    for i in self.openSet:
        tempDict[i] = self.fScore[i]
    minNode = min(tempDict, key=tempDict.get)
    return minNode

In [394]:
def get_neighbors(self, node):
    """Returns the neighbors of a node"""
    # TODO: Return the neighbors of a node
    return self.map.roads[node]

In [395]:
def get_gScore(self, node):
    """Returns the g Score of a node"""
    # TODO: Return the g Score of a node
    return self.gScore[node]

In [396]:
def get_tenative_gScore(self, current, neighbor):
    """Returns the tenative g Score of a node"""
    # TODO: Return the tenative  tentative g Score of the current node 
    # plus distance from the current node to it's neighbors
    # current node는 tuple이다. neighor 또한 tuple이다.
    return self.get_gScore(current) + self.distance(current, neighbor)

In [397]:
def is_open_empty(self):
    """returns True if the open set is empty. False otherwise. """
    # TODO: Return True if the open set is empty. False otherwise.
    if len(self.openSet) > 0:
        return False
    else:
        return True

In [398]:
def distance(self, node_1, node_2):
    """ Computes the Euclidean L2 Distance"""
    # TODO: Compute and return the Euclidean L2 Distance
    node_1_position = self.map.intersections[node_1]
    node_2_position = self.map.intersections[node_2]
    return math.sqrt((node_1_position[0]-node_2_position[0])**2 + (node_1_position[1] - node_2_position[1])**2)

In [399]:
def heuristic_cost_estimate(self, node):
    """ Returns the heuristic cost estimate of a node """
    # TODO: Return the heuristic cost estimate of a node
    return self.distance(node, self.goal)

In [400]:
def calculate_fscore(self, node):
    """Calculate the f score of a node. """
    # TODO: Calculate and returns the f score of a node. 
    # REMEMBER F = G + H
    return self.get_gScore(node) + self.heuristic_cost_estimate(node)

In [401]:
def record_best_path_to(self, current, neighbor):
    """Record the best path to a node """
    # TODO: Record the best path to a node, by updating cameFrom, gScore, and fScore
    
    self.cameFrom[neighbor] = current
    self.gScore[neighbor] = self.get_tenative_gScore(current, neighbor)
    self.fScore[neighbor] = self.calculate_fscore(neighbor)

In [402]:
PathPlanner.create_closedSet = create_closedSet
PathPlanner.create_openSet = create_openSet
PathPlanner.create_cameFrom = create_cameFrom
PathPlanner.create_gScore = create_gScore
PathPlanner.create_fScore = create_fScore
PathPlanner._reset = _reset
PathPlanner.set_map = set_map
PathPlanner.set_start = set_start
PathPlanner.set_goal = set_goal
PathPlanner.get_current_node = get_current_node
PathPlanner.get_neighbors = get_neighbors
PathPlanner.get_gScore = get_gScore
PathPlanner.get_tenative_gScore = get_tenative_gScore
PathPlanner.is_open_empty = is_open_empty
PathPlanner.distance = distance
PathPlanner.heuristic_cost_estimate = heuristic_cost_estimate
PathPlanner.calculate_fscore = calculate_fscore
PathPlanner.record_best_path_to = record_best_path_to

In [403]:
planner = PathPlanner(map_40, 5, 34)
path = planner.path
if path == [5, 16, 37, 12, 34]:
    print("great! Your code works for these inputs!")
else:
    print("something is off, your code produced the following:")
    print(path)

great! Your code works for these inputs!


### Testing your Code
If the code below produces no errors, your algorithm is behaving correctly. You are almost ready to submit! Before you submit, go through the following submission checklist:

**Submission Checklist**

1. Does my code pass all tests?
2. Does my code implement `A*` search and not some other search algorithm?
3. Do I use an **admissible heuristic** to direct search efforts towards the goal?
4. Do I use data structures which avoid unnecessarily slow lookups?

When you can answer "yes" to all of these questions, submit by pressing the Submit button in the lower right!

In [404]:
from test import test

test(PathPlanner)

All tests pass! Congratulations!


## Questions

**Instructions**  Answer the following questions in your own words. We do not you expect you to know all of this knowledge on the top of your head. We expect you to do research and ask question. However do not merely copy and paste the answer from a google or stackoverflow. Read the information and understand it first. Then use your own words to explain the answer.

- How would you explain A-Star to a family member(layman)?

** ANSWER **: 

A-star algorithm is a kind of search algorithm. It can find the shortest path between two nodes in a graph. It uses path length(g) and estimated distance to the goal(h). It finds the shortest length path while expanding minimum number of paths possible by minimizing both g and h. So we can denote estimated cost of the cheapest solution through node n like this : f(n) = g(n) + h(n). h(n) is heuristic function value and g(n) is the cost from node to node.

- How does A-Star search algorithm differ from Uniform cost search? What about Best First search?

** ANSWER **: 

Uniform cost search just expands out to the goal without specific directions. It expands out with same distance. In that case If search space is large, we can't find the goal quickly because It isn't directed towards the goal. On the other hand, A-star algorithm finds optimal path by using estimate distance to the goal, so it can find goal more quickly. 

Best First search also use estimate distance towards goal and takes closest path to the goal. But if there is an obstacle along the way, it sometimes take longer path since it always try to get closest path to the goal instead of passing by obstacle. In contrast, A-star algorithm can avoid this problem by taking adavantage of the total cost with both path cost and estimated distance towards goal. 

- What is a heuristic?

** ANSWER **: 

In informed search strategy, heuristic is estimated cost fo the cheapest path from the node n to the goal. For example in A-Star algorithm, we can use straight line distance between state node and the goal as heuristic. In general It is a technique designed to solve a problem in more efficient fashion. Often heuristic is used for approximating solutions.

- What is a consistent heuristic?

** ANSWER **: 

If the cost from the current node to a successor node plus the estimated cost from the successor node to the goal is less than or equal to the estimated cost from the current node to the goal, then It is the consistent heuristic. It satisfies the following inequality. 

h(n) <= c(n,n') + h(n') where n' is succeessor of node n. 

- What is a admissible heuristic? 

** ANSWER **:

An admissible heuristic is the heuristic function that never overestimate path cost to goal. For example, we can use straight line distance for g(n) in A-star algorithm because straight line distance is the shortest path between any two points. Threrefore f(n) never overestimate the true cost of a solution along the current path through n.

- ____ admissible heuristic are consistent.
*CHOOSE ONE*
    - All 
    - Some <== 
    - None
    
** ANSWER **: SOME

- ___ Consistent heuristic are admissible.
*CHOOSE ONE*
    - All <==
    - Some
    - None
    
** ANSWER **: ALL